### Problem
A company currently ships products from 5 plants to 4 warehouses. The company is considering the option of
closing down one or more plants. This would increase distribution cost but perhaps lower overall cost. What
plants, if any, should the company close?

Based on an example from Frontline Systems: http://www.solver.com/disfacility.htm

In [ ]:
# First, import packages
import pandas as pd
from IPython.display import Image 
import gurobipy as gp
from gurobipy import GRB

# Define a gurobipy model for the decision problem
m = gp.Model('facility')

In [ ]:
# Table 01: 
Image("facility.png", width=450, height=350)

### Data

In [ ]:
# Sets P and W, respectively
# When we code sets we can be more descriptive in the name
plants = ['P1','P2','P3','P4','P5']
warehouses = ['W1','W2','W3','W4']

In [ ]:
capacity = pd.Series([20, 22, 17, 19, 18], index = plants, name = "Plant Capacity")
capacity.to_frame()
#capacity

In [ ]:
demand = pd.Series([15, 18, 14, 20], index = warehouses, name = "Demand")
demand.to_frame()
#demand

In [ ]:
# Fixed costs for each plant
fixedCosts = pd.Series([12000, 15000, 17000, 13000, 16000], index = plants, name = "Fixed Cost")
fixedCosts.to_frame()

In [ ]:
# Transportation costs per thousand units
# Load annual lost interest cost data thorough dictionary command
transCosts = {    
    ('P1', 'W1'): 4000,
    ('P1', 'W2'): 2500,
    ('P1', 'W3'): 1200,
    ('P1', 'W4'): 2200,
    ('P2', 'W1'): 2000,
    ('P2', 'W2'): 2600,
    ('P2', 'W3'): 1800,
    ('P2', 'W4'): 2600, 
    ('P3', 'W1'): 3000,
    ('P3', 'W2'): 3400,
    ('P3', 'W3'): 2600,
    ('P3', 'W4'): 3100,
    ('P4', 'W1'): 2500,
    ('P4', 'W2'): 3000,
    ('P4', 'W3'): 4100,
    ('P4', 'W4'): 3700,
    ('P5', 'W1'): 4500,
    ('P5', 'W2'): 4000,
    ('P5', 'W3'): 3000,
    ('P5', 'W4'): 3200
}

### Decision Variable

In [ ]:
# Plant open decision variables: open[p] == 1 if plant p is open.
open = m.addVars(plants, vtype=GRB.BINARY, name="open")
m.update()
open

# Method-01: 
# open = m.addVars(plants, vtype=GRB.BINARY, obj=fixedCosts, name="open")

# Method-02:
# open = []
# for p in plants:
#    open.append(m.addVar(vtype=GRB.BINARY, obj=fixedCosts[p], name="open[%d]" % p))
# m.update()
# open

In [ ]:
# Transportation decision variables: transport[w,p] captures the
# optimal quantity to transport to warehouse w from plant p
transport = m.addVars(plants, warehouses, vtype=GRB.CONTINUOUS, name="transport")
m.update()
transport

# Method-01
# transport = m.addVars(warehouses, plants, obj=transCosts, name="trans")

# Method-02
# transport = []
# for w in warehouses:
#     transport.append([])
#     for p in plants:
#         transport[w].append(m.addVar(obj=transCosts[w][p],
#                                      name="trans[%d,%d]" % (w, p)))

### Constraints 

In [ ]:
# Capacity/Production constraints
# Note that the right-hand limit sets the production to zero if the plant is closed
#c1 = m.addConstrs((transport.sum("*", p) <= capacity[p] * open[p] for p in plants), name = "Capacity")
#m.update()
#c1


c1 = m.addConstrs((gp.quicksum(transport[p,w] for w in warehouses) <= capacity[p] * open[p] for p in plants), name = 'Capacity')
m.update()
c1

# Using Python looping constructs, the preceding would be...
#
# for p in plants:
#     m.addConstr(sum(transport[w][p] for w in warehouses)
#                 <= capacity[p] * open[p], "Capacity[%d]" % p)


In [ ]:
# Demand constraints
c2 = m.addConstrs((gp.quicksum(transport[p,w] for p in plants) == demand[w] for w in warehouses), name ="Demand")
m.update()
c2

# ... and the preceding would be ...
# for w in warehouses:
#     m.addConstr(sum(transport[w][p] for p in plants) == demand[w],
#                 "Demand[%d]" % w)

### Objective Function

- **Cost**: Minimize total costs.

In [ ]:
m.setObjective(gp.quicksum(transCosts[p,w]*transport[p,w] for p in plants for w in warehouses)+gp.quicksum(fixedCosts[p]*open[p] for p in plants),GRB.MINIMIZE)

In [ ]:
# The objective is to minimize the total fixed and variable costs
#m.ModelSense = GRB.MINIMIZE

In [ ]:
# Save model
m.write("facility.lp")

In [ ]:
# Solve
m.optimize()
    
# Print solution
print(f"\nTOTAL COSTS: {m.ObjVal:g}")
print("SOLUTION:")
for p in plants:
    if open[p].X > 0.99:
        print(f"Plant {p} open")
        for w in warehouses:
            if transport[p, w].X > 0:
                print(f"  Transport {transport[p, w].X:g} units to warehouse {w}")
    else:
        print(f"Plant {p} closed!")

-----------------------------------------------------